In [140]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import math
import time

In [41]:
# Defining paths to ride data 
RIDES = {'MK': {'DWARFS': '7_dwarfs_train.csv', 'PIRATES': 'pirates_of_caribbean.csv', 'SPLASH': 'splash_mountain.csv'},
         'EP': {'SOARIN': 'soarin.csv', 'SPACE': 'spaceship_earth.csv'},
         'HS': {'ALIEN': 'alien_saucers.csv', 'ROCKN': 'rock_n_rollercoaster.csv', 'SLINKY': 'slinky_dog.csv', 'TSM': 'toy_story_mania.csv'},
         'AK': {'FLIGHT': 'flight_of_passage.csv', 'DINO': 'dinosaur.csv', 'EVEREST': 'expedition_everest.csv', 'SAFARI': 'kilimanjaro_safaris.csv', 'NAVI': 'navi_river.csv'}
        }

In [49]:
# Defining "new" rides for exponential decay (opened in last 10 years)
NEW_RIDES = {'DWARFS': pd.Timestamp(2014, 5, 28), 
             'ALIEN': pd.Timestamp(2018, 6, 30), 'SLINKY': pd.Timestamp(2018, 6, 30), 
             'FLIGHT': pd.Timestamp(2017, 5, 27), 'NAVI': pd.Timestamp(2017, 5, 27)
            }

In [317]:
# Load in full datasets
mk = pd.read_csv('clean/mk.csv')
ep = pd.read_csv('clean/ep.csv')
hs = pd.read_csv('clean/hs.csv')
ak = pd.read_csv('clean/ak.csv')

mk["DATE"] = pd.to_datetime(mk["DATE"])
ep["DATE"] = pd.to_datetime(ep["DATE"])
hs["DATE"] = pd.to_datetime(hs["DATE"])
ak["DATE"] = pd.to_datetime(ak["DATE"])
mk["DATETIME"] = pd.to_datetime(mk["DATETIME"])
ep["DATETIME"] = pd.to_datetime(ep["DATETIME"])
hs["DATETIME"] = pd.to_datetime(hs["DATETIME"])
ak["DATETIME"] = pd.to_datetime(ak["DATETIME"])

In [60]:
ak.head()

,DATE,DATETIME,FLIGHT,DINO,EVEREST,SAFARI,NAVI
0,2015-01-01,2015-01-01 07:47:26,NaN,5.0,5.0,5.0,NaN
1,2015-01-01,2015-01-01 07:54:23,NaN,5.0,5.0,5.0,NaN
2,2015-01-01,2015-01-01 08:05:33,NaN,5.0,5.0,5.0,NaN
3,2015-01-01,2015-01-01 08:12:23,NaN,10.0,5.0,10.0,NaN
4,2015-01-01,2015-01-01 08:19:26,NaN,10.0,5.0,10.0,NaN


In [68]:
# Load in clusters
train = pd.read_csv('clean/train.csv')
test = pd.read_csv('clean/test.csv')

In [313]:
# Function to retroactively predict best path on a given day 
def computePath(park, date, data):
    paths = []
    start_time = []
    total_time = []
    full = data[data["DATE"]==date]
#     print(full)
    names = list(RIDES[park].keys())
    for ride in names:
        if ride in NEW_RIDES.keys():
            if NEW_RIDES[ride] > date:
                names.remove(ride)
                full = full.drop([ride], axis=1)
#     print(names)
    for i in range(min(30, len(full))):
        temp = full[names].iloc[i]
#         print(temp)
        if math.isnan(temp.min()):
            continue
        current_time = full['DATETIME'].iloc[i]
        rides = ''
        time = 0
        not_added = names.copy()
        for j in range(5):
            if math.isnan(temp.min()):
                break
            r = temp.idxmin()
            t = temp.min()
            if current_time + timedelta(minutes=t) > full['DATETIME'].iloc[-1]:
                break
            current_time = current_time + timedelta(minutes=t)
            rides = rides + r + ' '
            time = time + t
            not_added.remove(r)
            temp = full[full['DATETIME'] >= current_time].iloc[0][not_added].astype(float)
            while math.isnan(temp.min()) and j != 4:
                if current_time + timedelta(seconds=60) >= full['DATETIME'].iloc[-1]:
                    break
                current_time = current_time + timedelta(seconds=60)
                temp = full[full['DATETIME'] >= current_time].iloc[0][not_added].astype(float)
        if len(not_added) > 0:
            continue
        start_time.append(full['DATETIME'].iloc[i])
        paths.append(rides)
        total_time.append(time)
    for i in range(max(len(full)-20,0),len(full)):
        temp = full[names].iloc[i]
#         print(temp)
        if math.isnan(temp.min()):
            continue
        current_time = full['DATETIME'].iloc[i]
        rides = ''
        time = 0
        not_added = names.copy()
        for j in range(5):
            if math.isnan(temp.min()):
                break
            r = temp.idxmin()
            t = temp.min()
            if current_time + timedelta(minutes=t) > full['DATETIME'].iloc[-1]:
                break
            current_time = current_time + timedelta(minutes=t)
            rides = rides + r + ' '
            time = time + t
            not_added.remove(r)
            temp = full[full['DATETIME'] >= current_time].iloc[0][not_added].astype(float)
            while math.isnan(temp.min()) and j != 4:
                if current_time + timedelta(seconds=60) >= full['DATETIME'].iloc[-1]:
                    break
                current_time = current_time + timedelta(seconds=60)
                temp = full[full['DATETIME'] >= current_time].iloc[0][not_added].astype(float)
        if len(not_added) > 0:
            continue
        start_time.append(full['DATETIME'].iloc[i])
        paths.append(rides)
        total_time.append(time)
    idx = total_time.index(min(total_time))
    print(paths[idx])
    print(start_time[idx].time())
    print(total_time[idx])
    
    return paths[idx], start_time[idx], total_time[idx]

In [62]:
# TESTING
computePath('HS', pd.Timestamp(2015, 12, 15), hs)
computePath('MK', pd.Timestamp(2016, 12, 8), mk)

ROCKN TSM 
10:58:09
60.0
PIRATES SPLASH DWARFS 
20:21:10
30.0


In [78]:
mk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258436 entries, 0 to 258435
Data columns (total 5 columns):
DATE        258436 non-null datetime64[ns]
DATETIME    258436 non-null datetime64[ns]
DWARFS      258436 non-null float64
PIRATES     258418 non-null float64
SPLASH      258422 non-null float64
dtypes: datetime64[ns](2), float64(3)
memory usage: 9.9 MB


In [129]:
# Take averages across days in a dataset
def getClusterWaits(park_data, dates):
    data = park_data[park_data["DATE"].isin(dates)]
    data = data.drop(["DATE"], axis=1)
#     print(data)
    for i in range(len(data)):
        if data.iloc[i]["DATETIME"].hour >= 5:
            data.iat[i,0] = data.iloc[i]["DATETIME"].replace(year=2020, month=1, day=1, second=0)
        else:
            data.iat[i,0] = data.iloc[i]["DATETIME"].replace(year=2020, month=1, day=2, second=0)
    # Group by times and take averages
#     print(data)
    avg = data.groupby("DATETIME").mean()
    print(avg)
    return avg

In [131]:
#### Create array of datasets for training clusters 
mk_clusters = []
ep_clusters = []
hs_clusters = []
ak_clusters = []
start = time.time()
for i in range(35):
    print("CLUSTER", i)
    dates = train[train["CLUSTER"] == i]["DATE"]
    mk_clusters.append(getClusterWaits(mk, dates))
    ep_clusters.append(getClusterWaits(ep, dates))
    hs_clusters.append(getClusterWaits(hs, dates))
    ak_clusters.append(getClusterWaits(ak, dates))
    
end = time.time()
print("TIME TO COMPUTE CLUSTERS", end-start)
    

CLUSTER 0
                     DWARFS   PIRATES  SPLASH
DATETIME                                     
2020-01-01 07:40:00    50.0  5.000000     5.0
2020-01-01 07:49:00    30.0  5.000000     5.0
2020-01-01 07:50:00    45.0  8.333333     5.0
2020-01-01 07:55:00    45.0  7.500000     5.0
2020-01-01 07:56:00    30.0  5.000000     5.0
...                     ...       ...     ...
2020-01-02 01:19:00    30.0  5.000000     5.0
2020-01-02 01:29:00    37.5  5.000000     5.0
2020-01-02 01:40:00    15.0  5.000000     5.0
2020-01-02 01:47:00    20.0  5.000000     5.0
2020-01-02 01:53:00    30.0  5.000000     5.0

[609 rows x 3 columns]
                        SOARIN  SPACE
DATETIME                             
2020-01-01 07:51:00  10.000000    5.0
2020-01-01 07:56:00  10.000000    5.0
2020-01-01 07:57:00  10.000000    5.0
2020-01-01 08:01:00  10.000000    5.0
2020-01-01 08:06:00  10.000000    5.0
...                        ...    ...
2020-01-01 22:50:00  13.333333    5.0
2020-01-01 22:51:00  15.00

                        DWARFS    PIRATES    SPLASH
DATETIME                                           
2020-01-01 07:42:00  30.000000  10.000000  7.500000
2020-01-01 07:49:00  17.500000   7.500000  6.250000
2020-01-01 07:56:00  21.666667   6.666667  5.833333
2020-01-01 08:00:00  25.000000   7.500000  6.250000
2020-01-01 08:05:00  45.000000  15.000000  5.000000
...                        ...        ...       ...
2020-01-01 23:35:00  21.666667   6.666667  5.000000
2020-01-01 23:42:00  26.666667   6.666667  5.000000
2020-01-01 23:49:00  23.333333   6.666667  5.000000
2020-01-01 23:56:00  21.666667   5.000000  5.000000
2020-01-02 00:00:00  21.666667   5.000000  5.000000

[498 rows x 3 columns]
                        SOARIN  SPACE
DATETIME                             
2020-01-01 07:43:00  10.000000    5.0
2020-01-01 07:46:00  10.000000    5.0
2020-01-01 07:50:00  10.000000    5.0
2020-01-01 07:51:00  10.000000    5.0
2020-01-01 07:56:00  10.000000    5.0
...                        ...    

                     DWARFS  PIRATES  SPLASH
DATETIME                                    
2020-01-01 07:56:00  15.000     5.00     5.0
2020-01-01 08:00:00  15.000     5.00     5.0
2020-01-01 08:01:00  30.000     5.00     5.0
2020-01-01 08:04:00  30.000    10.00     5.0
2020-01-01 08:07:00  22.500     7.50     5.0
...                     ...      ...     ...
2020-01-01 23:56:00  31.875     6.25     5.0
2020-01-01 23:58:00  20.000     7.50     5.0
2020-01-02 00:00:00  35.000     5.00     5.0
2020-01-02 00:01:00  32.500     5.00     5.0
2020-01-02 00:02:00  30.000     5.00     5.0

[524 rows x 3 columns]
                        SOARIN  SPACE
DATETIME                             
2020-01-01 08:43:00  10.000000  5.000
2020-01-01 08:50:00  11.666667  5.000
2020-01-01 08:52:00  10.000000  5.000
2020-01-01 08:55:00  10.000000  5.000
2020-01-01 08:57:00  11.250000  6.875
...                        ...    ...
2020-01-01 23:30:00  10.000000  5.000
2020-01-01 23:36:00  10.000000  7.500
2020-01-01 

                     DWARFS  PIRATES  SPLASH
DATETIME                                    
2020-01-01 07:01:00    30.0      5.0     5.0
2020-01-01 07:04:00    30.0      5.0     5.0
2020-01-01 07:07:00    27.5      5.0     5.0
2020-01-01 07:13:00    35.0      5.0     5.0
2020-01-01 07:14:00    30.0      5.0     5.0
...                     ...      ...     ...
2020-01-02 00:27:00    30.0      5.0     5.0
2020-01-02 00:35:00    30.0      5.0     5.0
2020-01-02 00:42:00    30.0      5.0     5.0
2020-01-02 00:49:00    30.0      5.0     5.0
2020-01-02 00:56:00    50.0      5.0     5.0

[685 rows x 3 columns]
                        SOARIN  SPACE
DATETIME                             
2020-01-01 07:50:00  10.000000    5.0
2020-01-01 07:57:00  10.000000    5.0
2020-01-01 08:01:00  10.000000    5.0
2020-01-01 08:08:00  10.000000    5.0
2020-01-01 08:15:00  10.000000    5.0
...                        ...    ...
2020-01-01 22:40:00  15.000000    5.0
2020-01-01 22:43:00  20.000000    5.0
2020-01-01 

                     SOARIN      SPACE
DATETIME                              
2020-01-01 07:43:00    10.0   5.000000
2020-01-01 07:50:00    10.0   8.333333
2020-01-01 07:57:00    10.0   8.333333
2020-01-01 08:01:00    10.0   8.333333
2020-01-01 08:03:00    10.0  17.500000
...                     ...        ...
2020-01-01 22:41:00    10.0   5.000000
2020-01-01 22:46:00    10.0   5.000000
2020-01-01 22:51:00    10.0   5.000000
2020-01-01 22:56:00    10.0   5.000000
2020-01-01 23:01:00    10.0   5.000000

[461 rows x 2 columns]
                     ALIEN      ROCKN    SLINKY        TSM
DATETIME                                                  
2020-01-01 07:47:00    NaN   5.000000       NaN  12.500000
2020-01-01 07:52:00    NaN   5.000000       NaN  12.500000
2020-01-01 07:54:00    NaN   5.000000       NaN  20.000000
2020-01-01 07:57:00    NaN   5.000000       NaN  26.666667
2020-01-01 07:58:00    NaN  10.000000       NaN  25.000000
...                    ...        ...       ...        .

...

                     ALIEN  ROCKN  SLINKY   TSM
DATETIME                                       
2020-01-01 06:58:00   65.0   15.0    90.0   5.0
2020-01-01 07:02:00   65.0   15.0    90.0   5.0
2020-01-01 07:09:00   65.0   10.0    82.5   5.0
2020-01-01 07:16:00   65.0   10.0    97.5   5.0
2020-01-01 07:23:00   65.0   10.0    97.5   5.0
...                    ...    ...     ...   ...
2020-01-01 21:37:00    NaN   65.0     NaN  12.5
2020-01-01 21:44:00    NaN   65.0     NaN  12.5
2020-01-01 21:51:00    NaN   35.0     NaN  12.5
2020-01-01 21:58:00    NaN   25.0     NaN  12.5
2020-01-01 22:02:00    NaN   25.0     NaN  12.5

[475 rows x 4 columns]
                     FLIGHT       DINO   EVEREST    SAFARI       NAVI
DATETIME                                                             
2020-01-01 07:45:00     NaN  12.500000  5.000000  5.000000        NaN
2020-01-01 07:46:00    60.0   5.000000  5.000000  5.000000  15.000000
2020-01-01 07:52:00    90.0   6.590909  5.909091  6.818182  12.857143
20

                     FLIGHT  DINO  EVEREST  SAFARI   NAVI
DATETIME                                                 
2020-01-01 06:45:00  60.000   5.0      5.0    5.00   5.00
2020-01-01 06:52:00  60.000   5.0      5.0    5.00   5.00
2020-01-01 06:59:00  58.125   5.0      5.0    5.00   5.00
2020-01-01 07:03:00  60.000   5.0      5.0    5.00  20.00
2020-01-01 07:07:00  60.000   5.0      5.0    5.00   3.75
...                     ...   ...      ...     ...    ...
2020-01-02 00:38:00  78.750   5.0      5.0   11.25  13.75
2020-01-02 00:45:00  77.500   5.0      5.0   11.25  11.25
2020-01-02 00:52:00  61.250   5.0      5.0   11.25  11.25
2020-01-02 00:59:00  61.250   5.0      5.0   11.25  11.25
2020-01-02 01:03:00  61.250   5.0      5.0    5.00  11.25

[728 rows x 5 columns]
CLUSTER 25
                        DWARFS  PIRATES  SPLASH
DATETIME                                       
2020-01-01 07:46:00  30.000000      5.0     5.0
2020-01-01 07:49:00  21.666667      5.0     5.0
2020-01-01 07:50:00

                        DWARFS  PIRATES  SPLASH
DATETIME                                       
2020-01-01 07:00:00  40.000000      5.0     5.0
2020-01-01 07:01:00  26.666667      5.0     5.0
2020-01-01 07:05:00  45.000000      5.0     5.0
2020-01-01 07:07:00  27.000000      5.0     5.0
2020-01-01 07:08:00  45.000000      5.0     5.0
...                        ...      ...     ...
2020-01-02 01:35:00  45.000000      5.0     5.0
2020-01-02 01:42:00  45.000000      5.0     5.0
2020-01-02 01:49:00  37.500000      5.0     5.0
2020-01-02 01:56:00  33.750000      5.0     5.0
2020-01-02 02:00:00  36.666667      5.0     5.0

[688 rows x 3 columns]
                     SOARIN  SPACE
DATETIME                          
2020-01-01 07:43:00    10.0    5.0
2020-01-01 07:50:00    10.0    5.0
2020-01-01 07:57:00    10.0    5.0
2020-01-01 08:01:00    10.0    5.0
2020-01-01 08:08:00    13.0    6.0
...                     ...    ...
2020-01-01 22:36:00    30.0    5.0
2020-01-01 22:41:00    30.0    5.0
20

                        SOARIN  SPACE
DATETIME                             
2020-01-01 07:46:00  10.000000    5.0
2020-01-01 07:51:00  10.000000    5.0
2020-01-01 07:56:00  10.000000    5.0
2020-01-01 08:01:00  10.000000    5.0
2020-01-01 08:06:00  10.000000    5.0
...                        ...    ...
2020-01-01 22:39:00  30.000000    5.0
2020-01-01 22:43:00  19.166667    5.0
2020-01-01 22:50:00  16.666667    5.0
2020-01-01 22:57:00  17.000000    5.0
2020-01-01 23:01:00  16.666667    5.0

[400 rows x 2 columns]
                      ALIEN  ROCKN    SLINKY    TSM
DATETIME                                           
2020-01-01 06:58:00  6.2500   15.0  26.87500   5.00
2020-01-01 07:02:00  6.2500   15.0  26.87500   5.00
2020-01-01 07:09:00  5.0000    5.0  15.00000   7.50
2020-01-01 07:16:00  5.3125   10.0  16.71875   6.25
2020-01-01 07:23:00  5.0000    5.0  15.00000   7.50
...                     ...    ...       ...    ...
2020-01-01 23:37:00     NaN   40.0       NaN  13.75
2020-01-01 23:

                     SOARIN  SPACE
DATETIME                          
2020-01-01 07:46:00    10.0    5.0
2020-01-01 07:51:00    10.0    5.0
2020-01-01 07:56:00    10.0    5.0
2020-01-01 08:01:00    10.0    5.0
2020-01-01 08:06:00    10.0    5.0
...                     ...    ...
2020-01-01 22:50:00    10.0    5.0
2020-01-01 22:51:00    10.0    5.0
2020-01-01 22:56:00    10.0    5.0
2020-01-01 22:57:00    10.0    5.0
2020-01-01 23:01:00    10.0    5.0

[403 rows x 2 columns]
                       ALIEN      ROCKN  SLINKY        TSM
DATETIME                                                  
2020-01-01 07:16:00  7.81250  15.000000  42.500   5.000000
2020-01-01 07:23:00  6.40625  15.000000  23.750   5.000000
2020-01-01 07:37:00  6.25000  11.875000  21.250  11.250000
2020-01-01 07:42:00      NaN   5.000000     NaN  30.000000
2020-01-01 07:44:00  8.12500  11.666667  36.875   8.333333
...                      ...        ...     ...        ...
2020-01-01 23:51:00      NaN  25.000000     NaN  

In [208]:
# Fix cluster structure
for i, cluster in enumerate(ak_clusters):
    print(cluster)
    ak_clusters[i] = cluster.reset_index(level=["DATETIME"])


                     FLIGHT   DINO  EVEREST     SAFARI  NAVI
DATETIME                                                    
2020-01-01 07:45:00     NaN  20.00      5.0  30.000000   NaN
2020-01-01 07:52:00     NaN  20.00      5.0  30.000000   NaN
2020-01-01 07:58:00     NaN   5.00      5.0   5.000000   NaN
2020-01-01 07:59:00     NaN  12.50      5.0  13.333333   NaN
2020-01-01 08:03:00     NaN  11.25      5.0  13.333333   NaN
...                     ...    ...      ...        ...   ...
2020-01-01 23:38:00     NaN  10.00     10.0   5.000000   NaN
2020-01-01 23:43:00     NaN  10.00     10.0   5.000000   NaN
2020-01-01 23:48:00     NaN  10.00     10.0   5.000000   NaN
2020-01-01 23:53:00     NaN  10.00     10.0   5.000000   NaN
2020-01-01 23:58:00     NaN  10.00     10.0   5.000000   NaN

[387 rows x 5 columns]
                        FLIGHT      DINO   EVEREST    SAFARI       NAVI
DATETIME                                                               
2020-01-01 06:45:00  60.000000  5.00000

In [280]:
# Function to retroactively predict best path on a given day 
def predictPath(park, date, data):
    paths = []
    start_time = []
    total_time = []
    full = data.copy()
#     print(full)
    names = list(RIDES[park].keys())
    for ride in names:
        if ride in NEW_RIDES.keys():
            if NEW_RIDES[ride] > date:
                names.remove(ride)
                full = full.drop([ride], axis=1)
#     print(names)
    for i in range(min(30, len(full))):
        temp = full[names].iloc[i]
#         print(temp)
        if math.isnan(temp.min()):
            continue
        current_time = full['DATETIME'].iloc[i]
        rides = ''
        time = 0
        not_added = names.copy()
        for j in range(5):
            if math.isnan(temp.min()):
                break
            r = temp.idxmin()
            t = temp.min()
            if current_time + timedelta(minutes=t) > full['DATETIME'].iloc[-1]:
                break
            current_time = current_time + timedelta(minutes=t)
            rides = rides + r + ' '
            time = time + t
            not_added.remove(r)
            temp = full[full['DATETIME'] >= current_time].iloc[0][not_added].astype(float)
            while math.isnan(temp.min()) and j != 4:
                if current_time + timedelta(seconds=60) >= full['DATETIME'].iloc[-1]:
                    break
                current_time = current_time + timedelta(seconds=60)
                temp = full[full['DATETIME'] >= current_time].iloc[0][not_added].astype(float)
        if len(not_added) > 0:
            continue
        start_time.append(full['DATETIME'].iloc[i])
        paths.append(rides)
        total_time.append(time)
    for i in range(max(len(full)-20,0),len(full)):
        temp = full[names].iloc[i]
#         print(temp)
        if math.isnan(temp.min()):
            continue
        current_time = full['DATETIME'].iloc[i]
        rides = ''
        time = 0
        not_added = names.copy()
        for j in range(5):
            if math.isnan(temp.min()):
                break
            r = temp.idxmin()
            t = temp.min()
            if current_time + timedelta(minutes=t) > full['DATETIME'].iloc[-1]:
                break
            current_time = current_time + timedelta(minutes=t)
            rides = rides + r + ' '
            time = time + t
            not_added.remove(r)
            temp = full[full['DATETIME'] >= current_time].iloc[0][not_added].astype(float)
            while math.isnan(temp.min()) and j != 4:
                if current_time + timedelta(seconds=60) >= full['DATETIME'].iloc[-1]:
                    break
                current_time = current_time + timedelta(seconds=60)
                temp = full[full['DATETIME'] >= current_time].iloc[0][not_added].astype(float)
        if len(not_added) > 0:
            continue
        start_time.append(full['DATETIME'].iloc[i])
        paths.append(rides)
        total_time.append(time)

    idx = total_time.index(min(total_time))
    print(paths[idx])
    print(start_time[idx].time())
    print(total_time[idx])
    
    return paths[idx], start_time[idx], total_time[idx]

In [349]:
## Predict a path for one day
def predictMK(date, clusters, test, mk):
    c = test[test["DATE"] == date]["PREDS"].iloc[0]
    data = mk_clusters[c]
    opening = mk[mk["DATE"] == date].iloc[0]["DATETIME"].replace(year=2020, month=1, day=1)
    closing = mk[mk["DATE"] == date].iloc[-1]["DATETIME"].replace(year=2020, month=1, day=1)
    if closing.hour < 5:
        closing = closing.replace(day=2)
    data = data[(data["DATETIME"] >= opening) & (data["DATETIME"] <= closing)]
    data["DATETIME"] = pd.to_datetime(data["DATETIME"])
#     print(data)
    return predictPath("MK", pd.to_datetime(date), data)

def predictEP(date, clusters, test, ep):
    c = test[test["DATE"] == date]["PREDS"].iloc[0]
    data = ep_clusters[c]
    opening = ep[ep["DATE"] == date].iloc[0]["DATETIME"].replace(year=2020, month=1, day=1)
    closing = ep[ep["DATE"] == date].iloc[-1]["DATETIME"].replace(year=2020, month=1, day=1)
    if closing.hour < 5:
        closing = closing.replace(day=2)
    data = data[(data["DATETIME"] >= opening) & (data["DATETIME"] <= closing)]
    data["DATETIME"] = pd.to_datetime(data["DATETIME"])
#     print(data)
    return predictPath("EP", pd.to_datetime(date), data)

def predictHS(date, clusters, test, hs):
    c = test[test["DATE"] == date]["PREDS"].iloc[0]
    data = hs_clusters[c]
    opening = hs[hs["DATE"] == date].iloc[0]["DATETIME"].replace(year=2020, month=1, day=1)
    closing = hs[hs["DATE"] == date].iloc[-1]["DATETIME"].replace(year=2020, month=1, day=1)
    if closing.hour < 5:
        closing = closing.replace(day=2)
    data = data[(data["DATETIME"] >= opening) & (data["DATETIME"] <= closing)]
    data["DATETIME"] = pd.to_datetime(data["DATETIME"])
#     print(data)
    return predictPath("HS", pd.to_datetime(date), data)
    
def predictAK(date, clusters, test, ak):
    c = test[test["DATE"] == date]["PREDS"].iloc[0]
    data = ak_clusters[c]
    opening = ak[ak["DATE"] == date].iloc[0]["DATETIME"].replace(year=2020, month=1, day=1)
    closing = ak[ak["DATE"] == date].iloc[-1]["DATETIME"].replace(year=2020, month=1, day=1)
    if closing.hour < 5:
        closing = closing.replace(day=2)
    data = data[(data["DATETIME"] >= opening) & (data["DATETIME"] <= closing)]
    data["DATETIME"] = pd.to_datetime(data["DATETIME"])
#     print(data)
    return predictPath("AK", pd.to_datetime(date), data)

In [267]:
idx = np.random.randint(len(test))
date = test.iloc[idx]["DATE"]
print(date)
predictMK(date, mk_clusters)

2019-11-25


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


PIRATES SPLASH DWARFS 
09:10:00
82.83430232558139


('PIRATES SPLASH DWARFS ', Timestamp('2020-01-01 09:10:00'), 82.83430232558139)

In [224]:
computePath('MK', pd.to_datetime(test.iloc[idx]["DATE"]), mk)

PIRATES SPLASH DWARFS 
07:48:02
25.0


In [214]:
mk_clusters[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609 entries, 0 to 608
Data columns (total 4 columns):
DATETIME    609 non-null datetime64[ns]
DWARFS      609 non-null float64
PIRATES     609 non-null float64
SPLASH      609 non-null float64
dtypes: datetime64[ns](1), float64(3)
memory usage: 19.2 KB


In [327]:
## Accuracy 
def isAccurate(pred_path, pred_start, actual_total, date, data, thresh):
    rides = pred_path.split()
    current_time = pred_start.replace(year=date.year, month=date.month, day=date.day)
    comp_time = 0
    for ride in rides:
        i = data.index[data['DATETIME'] >= current_time].tolist()
        t = data.iloc[i[0]][ride]
        j = 1
        while math.isnan(t):
            t = data.iloc[i[j]][ride]
            j += 1
        current_time = current_time + timedelta(minutes=t)
        comp_time = comp_time + t
    print("Comp time:", comp_time)
    return(abs(comp_time - actual_total) <= thresh)
    

In [250]:
## TESTING
pred_path = "PIRATES SPLASH DWARFS "
pred_start = pd.Timestamp(2020, 1, 1, 8, 25)
actual_total = 25.0

isAccurate(pred_path, pred_start, actual_total, pd.Timestamp(2019, 8, 6), mk)

Comp time: 25.0


True

In [299]:
# Script to iterate through subset of test in MK and compute accuracy
mk_count_acc30 = 0
mk_count_acc25 = 0
mk_count_acc20 = 0
mk_count_acc15 = 0
mk_count_acc45 = 0
mk_count_acc60 = 0
idx = np.random.randint(0,len(test),30)
sub = test.iloc[idx,:]
for date in sub["DATE"]:
    # Compute actual pred
    print("#####", date, "#####")
    print("ACTUAL")
    act_path, act_start, act_total = computePath('MK', pd.to_datetime(date), mk)
    print("PREDICT")
    pred_path, pred_start, pred_total = predictMK(date, mk_clusters)
    mk_count_acc30 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), mk, 30)
    mk_count_acc25 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), mk, 25)
    mk_count_acc20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), mk, 20)
    mk_count_acc15 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), mk, 15)
    mk_count_acc45 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), mk, 45)
    mk_count_acc60 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), mk, 60)

##### 2016-08-21 #####
ACTUAL
DWARFS PIRATES SPLASH 
08:42:20
15.0
PREDICT


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


SPLASH PIRATES DWARFS 
08:41:00
17.604166666666668
Comp time: 42.5
Comp time: 42.5
Comp time: 42.5
Comp time: 42.5
Comp time: 42.5
Comp time: 42.5
##### 2018-09-07 #####
ACTUAL
PIRATES SPLASH DWARFS 
22:27:07
30.0
PREDICT
PIRATES SPLASH DWARFS 
23:23:00
22.5
Comp time: 35.0
Comp time: 35.0
Comp time: 35.0
Comp time: 35.0
Comp time: 35.0
Comp time: 35.0
##### 2017-08-16 #####
ACTUAL
SPLASH PIRATES DWARFS 
09:00:11
75.0
PREDICT
SPLASH PIRATES DWARFS 
08:58:00
47.5
Comp time: 75.0
Comp time: 75.0
Comp time: 75.0
Comp time: 75.0
Comp time: 75.0
Comp time: 75.0
##### 2016-05-25 #####
ACTUAL
PIRATES SPLASH DWARFS 
08:42:07
40.0
PREDICT
PIRATES SPLASH DWARFS 
09:26:00
10.0
Comp time: 75.0
Comp time: 75.0
Comp time: 75.0
Comp time: 75.0
Comp time: 75.0
Comp time: 75.0
##### 2015-09-02 #####
ACTUAL
PIRATES SPLASH DWARFS 
08:49:16
40.0
PREDICT
PIRATES SPLASH DWARFS 
08:53:00
40.0
Comp time: 40.0
Comp time: 40.0
Comp time: 40.0
Comp time: 40.0
Comp time: 40.0
Comp time: 40.0
##### 2017-02-04 ####

In [300]:
mk_acc30.append(mk_count_acc30 / 30)
mk_acc25.append(mk_count_acc25 / 30)
mk_acc20.append(mk_count_acc20 / 30)
mk_acc15.append(mk_count_acc15 / 30)
mk_acc45.append(mk_count_acc45 / 30)
mk_acc60.append(mk_count_acc60 / 30)

print(mk_acc30)
print(mk_acc25)
print(mk_acc20)
print(mk_acc15)
print(mk_acc45)
print(mk_acc60)

[0.8666666666666667, [0.7666666666666667], 0.7333333333333333]
[0.7666666666666667, [0.7333333333333333], 0.5666666666666667]
[0.6, [0.7], 0.5666666666666667]
[0.6, [0.7], 0.5666666666666667]
[0.9, [0.9], 0.9]
[0.9333333333333333, [0.9666666666666667], 0.9]


In [302]:
# Script to iterate through subset of test in EP and compute accuracy
ep_count_acc30 = 0
ep_count_acc25 = 0
ep_count_acc20 = 0
ep_count_acc15 = 0
ep_count_acc45 = 0
ep_count_acc60 = 0
idx = np.random.randint(0,len(test),30)
sub = test.iloc[idx,:]
for date in sub["DATE"]:
    # Compute actual pred
    print("#####", date, "#####")
    print("ACTUAL")
    act_path, act_start, act_total = computePath('EP', pd.to_datetime(date), ep)
    print("PREDICT")
    pred_path, pred_start, pred_total = predictEP(date, ep_clusters)
    ep_count_acc30 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ep, 30)
    ep_count_acc25 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ep, 25)
    ep_count_acc20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ep, 20)
    ep_count_acc15 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ep, 15)
    ep_count_acc45 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ep, 45)
    ep_count_acc60 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ep, 60)

##### 2016-10-09 #####
ACTUAL
SPACE SOARIN 
08:43:10
15.0
PREDICT


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


SPACE SOARIN 
08:54:00
15.0
Comp time: 30.0
Comp time: 30.0
Comp time: 30.0
Comp time: 30.0
Comp time: 30.0
Comp time: 30.0
##### 2019-03-25 #####
ACTUAL
SPACE SOARIN 
08:50:08
10.0
PREDICT
SPACE SOARIN 
08:56:00
21.875
Comp time: 10.0
Comp time: 10.0
Comp time: 10.0
Comp time: 10.0
Comp time: 10.0
Comp time: 10.0
##### 2016-06-14 #####
ACTUAL
SPACE SOARIN 
21:08:11
65.0
PREDICT
SOARIN SPACE 
09:02:00
20.0
Comp time: 185.0
Comp time: 185.0
Comp time: 185.0
Comp time: 185.0
Comp time: 185.0
Comp time: 185.0
##### 2019-07-04 #####
ACTUAL
SPACE SOARIN 
07:50:02
15.0
PREDICT
SOARIN SPACE 
20:37:00
11.333333333333332
Comp time: 35.0
Comp time: 35.0
Comp time: 35.0
Comp time: 35.0
Comp time: 35.0
Comp time: 35.0
##### 2018-07-03 #####
ACTUAL
SPACE SOARIN 
08:57:06
15.0
PREDICT
SOARIN SPACE 
20:37:00
11.333333333333332
Comp time: 15.0
Comp time: 15.0
Comp time: 15.0
Comp time: 15.0
Comp time: 15.0
Comp time: 15.0
##### 2015-08-13 #####
ACTUAL
SPACE SOARIN 
20:43:05
10.0
PREDICT
SPACE SOARIN 


In [303]:
ep_acc30.append(ep_count_acc30 / 30)
ep_acc25.append(ep_count_acc25 / 30)
ep_acc20.append(ep_count_acc20 / 30)
ep_acc15.append(ep_count_acc15 / 30)
ep_acc45.append(ep_count_acc45 / 30)
ep_acc60.append(ep_count_acc60 / 30)

print(ep_acc30)
print(ep_acc25)
print(ep_acc20)
print(ep_acc15)
print(ep_acc45)
print(ep_acc60)

[0.8333333333333334, [0.9], 0.8666666666666667]
[0.8333333333333334, [0.8333333333333334], 0.8333333333333334]
[0.8, [0.8], 0.8333333333333334]
[0.6666666666666666, [0.8], 0.7333333333333333]
[0.9333333333333333, [0.9333333333333333], 0.9333333333333333]
[0.9333333333333333, [0.9666666666666667], 0.9333333333333333]


In [304]:
# Script to iterate through subset of test in EP and compute accuracy
hs_count_acc30 = 0
hs_count_acc25 = 0
hs_count_acc20 = 0
hs_count_acc15 = 0
hs_count_acc45 = 0
hs_count_acc60 = 0
idx = np.random.randint(0,len(test),30)
sub = test.iloc[idx,:]
for date in sub["DATE"]:
    # Compute actual pred
    print("#####", date, "#####")
    print("ACTUAL")
    act_path, act_start, act_total = computePath('HS', pd.to_datetime(date), hs)
    print("PREDICT")
    pred_path, pred_start, pred_total = predictHS(date, hs_clusters)
    hs_count_acc30 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), hs, 30)
    hs_count_acc25 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), hs, 25)
    hs_count_acc20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), hs, 20)
    hs_count_acc15 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), hs, 15)
    hs_count_acc45 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), hs, 45)
    hs_count_acc60 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), hs, 60)

##### 2016-02-14 #####
ACTUAL
ROCKN TSM 
06:58:08
40.0
PREDICT


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


TSM ROCKN 
07:03:00
20.0
Comp time: 50.0
Comp time: 50.0
Comp time: 50.0
Comp time: 50.0
Comp time: 50.0
Comp time: 50.0
##### 2015-10-02 #####
ACTUAL
ROCKN TSM 
08:58:08
30.0
PREDICT
ROCKN TSM 
09:02:00
38.958333333333336
Comp time: 70.0
Comp time: 70.0
Comp time: 70.0
Comp time: 70.0
Comp time: 70.0
Comp time: 70.0
##### 2017-08-12 #####
ACTUAL
ROCKN TSM 
08:51:10
15.0
PREDICT
ROCKN TSM 
08:53:00
36.66666666666667
Comp time: 15.0
Comp time: 15.0
Comp time: 15.0
Comp time: 15.0
Comp time: 15.0
Comp time: 15.0
##### 2017-07-25 #####
ACTUAL
ROCKN TSM 
08:51:15
25.0
PREDICT
ROCKN TSM 
08:52:00
29.0625
Comp time: 30.0
Comp time: 30.0
Comp time: 30.0
Comp time: 30.0
Comp time: 30.0
Comp time: 30.0
##### 2015-04-30 #####
ACTUAL
ROCKN TSM 
08:57:11
25.0
PREDICT
ROCKN TSM 
09:08:00
53.75
Comp time: 60.0
Comp time: 60.0
Comp time: 60.0
Comp time: 60.0
Comp time: 60.0
Comp time: 60.0
##### 2017-10-02 #####
ACTUAL
ROCKN TSM 
08:51:08
15.0
PREDICT
ROCKN TSM 
09:01:00
20.0
Comp time: 35.0
Comp tim

In [305]:
hs_acc30.append(hs_count_acc30 / 30)
hs_acc25.append(hs_count_acc25 / 30)
hs_acc20.append(hs_count_acc20 / 30)
hs_acc15.append(hs_count_acc15 / 30)
hs_acc45.append(hs_count_acc45 / 30)
hs_acc60.append(hs_count_acc60 / 30)

print(hs_acc30)
print(hs_acc25)
print(hs_acc20)
print(hs_acc15)
print(hs_acc45)
print(hs_acc60)

[0.5666666666666667, 0.7666666666666667, 0.6333333333333333]
[0.5333333333333333, 0.7, 0.6]
[0.5333333333333333, 0.7, 0.6]
[0.4666666666666667, 0.6, 0.4666666666666667]
[0.6666666666666666, 0.8, 0.7]
[0.7666666666666667, 0.8666666666666667, 0.7666666666666667]


In [306]:
# Script to iterate through subset of test in EP and compute accuracy
ak_count_acc30 = 0
ak_count_acc25 = 0
ak_count_acc20 = 0
ak_count_acc15 = 0
ak_count_acc45 = 0
ak_count_acc60 = 0
idx = np.random.randint(0,len(test),30)
sub = test.iloc[idx,:]
for date in sub["DATE"]:
    # Compute actual pred
    print("#####", date, "#####")
    print("ACTUAL")
    act_path, act_start, act_total = computePath('AK', pd.to_datetime(date), ak)
    print("PREDICT")
    pred_path, pred_start, pred_total = predictAK(date, ak_clusters)
    ak_count_acc30 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ak, 30)
    ak_count_acc25 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ak, 25)
    ak_count_acc20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ak, 20)
    ak_count_acc15 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ak, 15)
    ak_count_acc45 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ak, 45)
    ak_count_acc60 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ak, 60)

##### 2017-08-30 #####
ACTUAL
DINO EVEREST SAFARI NAVI FLIGHT 
08:44:51
120.0
PREDICT


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


EVEREST DINO SAFARI NAVI FLIGHT 
08:46:00
90.58510638297872
Comp time: 120.0
Comp time: 120.0
Comp time: 120.0
Comp time: 120.0
Comp time: 120.0
Comp time: 120.0
##### 2018-10-17 #####
ACTUAL
NAVI DINO EVEREST SAFARI FLIGHT 
08:30:10
67.92723352971346
PREDICT
EVEREST DINO SAFARI NAVI FLIGHT 
08:52:00
173.4839015151515
Comp time: 150.0
Comp time: 150.0
Comp time: 150.0
Comp time: 150.0
Comp time: 150.0
Comp time: 150.0
##### 2019-01-23 #####
ACTUAL
DINO EVEREST SAFARI NAVI FLIGHT 
08:30:16
145.0
PREDICT
DINO SAFARI EVEREST NAVI FLIGHT 
08:33:00
227.70833333333334
Comp time: 145.0
Comp time: 145.0
Comp time: 145.0
Comp time: 145.0
Comp time: 145.0
Comp time: 145.0
##### 2017-11-23 #####
ACTUAL
DINO EVEREST SAFARI NAVI FLIGHT 
07:46:19
195.0
PREDICT
DINO SAFARI EVEREST NAVI FLIGHT 
07:55:00
117.0
Comp time: 260.0
Comp time: 260.0
Comp time: 260.0
Comp time: 260.0
Comp time: 260.0
Comp time: 260.0
##### 2016-07-23 #####
ACTUAL
DINO EVEREST SAFARI 
07:39:44
15.0
PREDICT
DINO EVEREST SAFARI 

In [307]:
ak_acc30.append(ak_count_acc30 / 30)
ak_acc25.append(ak_count_acc25 / 30)
ak_acc20.append(ak_count_acc20 / 30)
ak_acc15.append(ak_count_acc15 / 30)
ak_acc45.append(ak_count_acc45 / 30)
ak_acc60.append(ak_count_acc60 / 30)

print(ak_acc30)
print(ak_acc25)
print(ak_acc20)
print(ak_acc15)
print(ak_acc45)
print(ak_acc60)

[0.7333333333333333, 0.6, 0.6666666666666666]
[0.7, 0.5666666666666667, 0.6666666666666666]
[0.6333333333333333, 0.5666666666666667, 0.6666666666666666]
[0.6333333333333333, 0.43333333333333335, 0.6]
[0.8333333333333334, 0.7, 0.6666666666666666]
[0.8333333333333334, 0.8, 0.7666666666666667]


In [401]:
## 2020!!!
# Load in full datasets
mk20 = pd.read_csv('clean/mk20.csv')
ep20 = pd.read_csv('clean/ep20.csv')
hs20 = pd.read_csv('clean/hs20.csv')
ak20 = pd.read_csv('clean/ak20.csv')

mk20["DATE"] = pd.to_datetime(mk20["DATE"])
ep20["DATE"] = pd.to_datetime(ep20["DATE"])
hs20["DATE"] = pd.to_datetime(hs20["DATE"])
ak20["DATE"] = pd.to_datetime(ak20["DATE"])
mk20["DATETIME"] = pd.to_datetime(mk20["DATETIME"])
ep20["DATETIME"] = pd.to_datetime(ep20["DATETIME"])
hs20["DATETIME"] = pd.to_datetime(hs20["DATETIME"])
ak20["DATETIME"] = pd.to_datetime(ak20["DATETIME"])

# Load in clusters
test20 = pd.read_csv('clean/test20.csv')

In [404]:
# Script to iterate through subset of test in MK and compute accuracy
mk_count_acc30_20 = 0
mk_count_acc25_20 = 0
mk_count_acc20_20 = 0
mk_count_acc15_20 = 0
mk_count_acc45_20 = 0
mk_count_acc60_20 = 0
idx = np.random.randint(0,len(test20),20)
sub = test20.iloc[idx,:]
for date in sub["DATE"]:
    # Compute actual pred
    print("#####", date, "#####")
    print("ACTUAL")
    act_path, act_start, act_total = computePath('MK', pd.to_datetime(date), mk20)
    print("PREDICT")
    pred_path, pred_start, pred_total = predictMK(date, mk_clusters, test20, mk20)
    mk_count_acc30_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), mk20, 30)
    mk_count_acc25_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), mk20, 25)
    mk_count_acc20_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), mk20, 20)
    mk_count_acc15_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), mk20, 15)
    mk_count_acc45_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), mk20, 45)
    mk_count_acc60_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), mk20, 60)

##### 2020-11-19 #####
ACTUAL
PIRATES SPLASH DWARFS 
09:26:01
72.5
PREDICT


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


PIRATES SPLASH DWARFS 
09:19:00
56.5
Comp time: 75.0
Comp time: 75.0
Comp time: 75.0
Comp time: 75.0
Comp time: 75.0
Comp time: 75.0
##### 2020-11-11 #####
ACTUAL
PIRATES SPLASH DWARFS 
09:19:02
70.0
PREDICT
PIRATES SPLASH DWARFS 
09:19:00
56.5
Comp time: 70.0
Comp time: 70.0
Comp time: 70.0
Comp time: 70.0
Comp time: 70.0
Comp time: 70.0
##### 2020-12-01 #####
ACTUAL
SPLASH PIRATES DWARFS 
09:31:21
63.75
PREDICT
SPLASH PIRATES DWARFS 
09:23:00
58.75
Comp time: 68.75
Comp time: 68.75
Comp time: 68.75
Comp time: 68.75
Comp time: 68.75
Comp time: 68.75
##### 2020-12-17 #####
ACTUAL
PIRATES SPLASH DWARFS 
08:40:02
55.0
PREDICT
SPLASH PIRATES DWARFS 
08:09:00
45.0
Comp time: 80.0
Comp time: 80.0
Comp time: 80.0
Comp time: 80.0
Comp time: 80.0
Comp time: 80.0
##### 2020-12-28 #####
ACTUAL
PIRATES DWARFS SPLASH 
07:51:21
80.0
PREDICT
PIRATES SPLASH DWARFS 
07:59:00
42.5
Comp time: 107.6171875
Comp time: 107.6171875
Comp time: 107.6171875
Comp time: 107.6171875
Comp time: 107.6171875
Comp tim

ValueError: min() arg is an empty sequence

In [ ]:
mk_acc30_20=[]
mk_acc25_20=[]
mk_acc20_20=[]
mk_acc15_20=[]
mk_acc45_20=[]
mk_acc60_20=[]

mk_acc30_20.append(mk_count_acc30_20 / 20)
mk_acc25_20.append(mk_count_acc25_20 / 20)
mk_acc20_20.append(mk_count_acc20_20 / 20)
mk_acc15_20.append(mk_count_acc15_20 / 20)
mk_acc45_20.append(mk_count_acc45_20 / 20)
mk_acc60_20.append(mk_count_acc60_20 / 20)

print(mk_acc30_20)
print(mk_acc25_20)
print(mk_acc20_20)
print(mk_acc15_20)
print(mk_acc45_20)
print(mk_acc60_20)

In [ ]:
# Script to iterate through subset of test in MK and compute accuracy
ep_count_acc30_20 = 0
ep_count_acc25_20 = 0
ep_count_acc20_20 = 0
ep_count_acc15_20 = 0
ep_count_acc45_20 = 0
ep_count_acc60_20 = 0
idx = np.random.randint(0,len(test20),20)
sub = test20.iloc[idx,:]
for date in sub["DATE"]:
    # Compute actual pred
    print("#####", date, "#####")
    print("ACTUAL")
    act_path, act_start, act_total = computePath('EP', pd.to_datetime(date), ep20)
    print("PREDICT")
    pred_path, pred_start, pred_total = predictEP(date, mk_clusters, test20, ep20)
    ep_count_acc30_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ep20, 30)
    ep_count_acc25_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ep20, 25)
    ep_count_acc20_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ep20, 20)
    ep_count_acc15_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ep20, 15)
    ep_count_acc45_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ep20, 45)
    ep_count_acc60_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ep20, 60)

In [ ]:
ep_acc30_20=[]
ep_acc25_20=[]
ep_acc20_20=[]
ep_acc15_20=[]
ep_acc45_20=[]
ep_acc60_20=[]

ep_acc30_20.append(ep_count_acc30_20 / 20)
ep_acc25_20.append(ep_count_acc25_20 / 20)
ep_acc20_20.append(ep_count_acc20_20 / 20)
ep_acc15_20.append(ep_count_acc15_20 / 20)
ep_acc45_20.append(ep_count_acc45_20 / 20)
ep_acc60_20.append(ep_count_acc60_20 / 20)

print(ep_acc30_20)
print(ep_acc25_20)
print(ep_acc20_20)
print(ep_acc15_20)
print(ep_acc45_20)
print(ep_acc60_20)

In [ ]:
# Script to iterate through subset of test in MK and compute accuracy
hs_count_acc30_20 = 0
hs_count_acc25_20 = 0
hs_count_acc20_20 = 0
hs_count_acc15_20 = 0
hs_count_acc45_20 = 0
hs_count_acc60_20 = 0
idx = np.random.randint(0,len(test20),20)
sub = test20.iloc[idx,:]
for date in sub["DATE"]:
    # Compute actual pred
    print("#####", date, "#####")
    print("ACTUAL")
    act_path, act_start, act_total = computePath('HS', pd.to_datetime(date), hs20)
    print("PREDICT")
    pred_path, pred_start, pred_total = predictHS(date, mk_clusters, test20, hs20)
    hs_count_acc30_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), hs20, 30)
    hs_count_acc25_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), hs20, 25)
    hs_count_acc20_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), hs20, 20)
    hs_count_acc15_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), hs20, 15)
    hs_count_acc45_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), hs20, 45)
    hs_count_acc60_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), hs20, 60)

In [ ]:
hs_acc30_20=[]
hs_acc25_20=[]
hs_acc20_20=[]
hs_acc15_20=[]
hs_acc45_20=[]
hs_acc60_20=[]

hs_acc30_20.append(hs_count_acc30_20 / 20)
hs_acc25_20.append(hs_count_acc25_20 / 20)
hs_acc20_20.append(hs_count_acc20_20 / 20)
hs_acc15_20.append(hs_count_acc15_20 / 20)
hs_acc45_20.append(hs_count_acc45_20 / 20)
hs_acc60_20.append(hs_count_acc60_20 / 20)

print(hs_acc30_20)
print(hs_acc25_20)
print(hs_acc20_20)
print(hs_acc15_20)
print(hs_acc45_20)
print(hs_acc60_20)

In [ ]:
# Script to iterate through subset of test in MK and compute accuracy
ak_count_acc30_20 = 0
ak_count_acc25_20 = 0
ak_count_acc20_20 = 0
ak_count_acc15_20 = 0
ak_count_acc45_20 = 0
ak_count_acc60_20 = 0
idx = np.random.randint(0,len(test20),20)
sub = test20.iloc[idx,:]
for date in sub["DATE"]:
    # Compute actual pred
    print("#####", date, "#####")
    print("ACTUAL")
    act_path, act_start, act_total = computePath('AK', pd.to_datetime(date), ak20)
    print("PREDICT")
    pred_path, pred_start, pred_total = predictAK(date, mk_clusters, test20, ak20)
    ak_count_acc30_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ak20, 30)
    ak_count_acc25_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ak20, 25)
    ak_count_acc20_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ak20, 20)
    ak_count_acc15_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ak20, 15)
    ak_count_acc45_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ak20, 45)
    ak_count_acc60_20 += isAccurate(pred_path, pred_start, act_total, pd.to_datetime(date), ak20, 60)

In [ ]:
ak_acc30_20=[]
ak_acc25_20=[]
ak_acc20_20=[]
ak_acc15_20=[]
ak_acc45_20=[]
ak_acc60_20=[]

ak_acc30_20.append(ak_count_acc30_20 / 20)
ak_acc25_20.append(ak_count_acc25_20 / 20)
ak_acc20_20.append(ak_count_acc20_20 / 20)
ak_acc15_20.append(ak_count_acc15_20 / 20)
ak_acc45_20.append(ak_count_acc45_20 / 20)
ak_acc60_20.append(ak_count_acc60_20 / 20)

print(ak_acc30_20)
print(ak_acc25_20)
print(ak_acc20_20)
print(ak_acc15_20)
print(ak_acc45_20)
print(ak_acc60_20)